## Analisi distribuzione classi della cartella Spettrogrammi per riduzione della dimensionalità dei dati

In [ ]:
import numpy as np
import utils 

In [ ]:
df_pathsTarget, df_namesTarget = utils.create_dataframe_from_files('Spettrogrammi/Target')
df_pathsNonTarget, df_namesNonTarget = utils.create_dataframe_from_files('Spettrogrammi/Non-Target')

import os

# Carica il DataFrame da df_paths.csv
# df_paths = pd.read_csv('Dataset/df_paths.csv', header=None, names=['path'])

# Crea una copia del DataFrame originale
df_pathsTargetCopy = df_pathsTarget.copy()
df_pathsNonTargetCopy = df_pathsNonTarget.copy()

# Estrai la classe e il target dai percorsi dei file
df_pathsTargetCopy['class'] = df_pathsTargetCopy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(path)))
df_pathsTargetCopy['target'] = df_pathsTargetCopy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(os.path.dirname(path))))

df_pathsNonTargetCopy['class'] = df_pathsNonTargetCopy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(path)))
df_pathsNonTargetCopy['target'] = df_pathsNonTargetCopy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(os.path.dirname(path))))

# Target

In [ ]:
utils.plot_class_distribution_horizontal(df_pathsTargetCopy, 'class')

# Non Target

In [ ]:
utils.plot_class_distribution_horizontal(df_pathsNonTargetCopy, 'class')


## Eliminati dal csv i path relativi a cartelle con meno di 5 file

In [ ]:
import pandas as pd
import os

def filter_csv_by_file_count(csv_path, k, output_csv_path):
    df = pd.read_csv(csv_path, header=None, names=['FilePath'])

    df['Subfolder'] = df['FilePath'].apply(lambda path: os.path.dirname(path))
    grouped_df = df.groupby('Subfolder').size().reset_index(name='Count')

    filtered_grouped_df = grouped_df[grouped_df['Count'] >= k]

    filtered_df = pd.merge(df, filtered_grouped_df, how='inner', on='Subfolder')

    filtered_df = filtered_df.drop(columns=['Count', 'Subfolder'])

    filtered_df.to_csv(output_csv_path, index=False, header=False)

filter_csv_by_file_count('Spettrogrammi/Target/df_paths.csv', 5, 'Spettrogrammi/Target/df_paths_filtered.csv')

In [ ]:
df_filtered = pd.read_csv('Spettrogrammi/Target/df_paths_filtered.csv', header=None, names=['FilePath'])
df_filtered_copy = df_filtered.copy()

df_filtered_copy['class'] = df_filtered_copy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(path)))
df_filtered_copy['target'] = df_filtered_copy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(os.path.dirname(path))))
utils.plot_class_distribution_horizontal(df_filtered_copy, 'class')


In [ ]:
filter_csv_by_file_count('Spettrogrammi/Non-Target/df_paths.csv', 5, 'Spettrogrammi/Non-Target/df_paths_filtered.csv')


In [ ]:
df_filtered = pd.read_csv('Spettrogrammi/Non-Target/df_paths_filtered.csv', header=None, names=['FilePath'])
df_filtered_copy = df_filtered.copy()

df_filtered_copy['class'] = df_filtered_copy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(path)))
df_filtered_copy['target'] = df_filtered_copy['FilePath'].apply(lambda path: os.path.basename(os.path.dirname(os.path.dirname(path))))
utils.plot_class_distribution_horizontal(df_filtered_copy, 'class')